# Tratamento de dados - Secretaria de segurança do estado do Paraná

## Descrição do notebook

O objetivo desse notebook é realizar o tratamento dos dados fornecidos pela Secretaria de Segurança Pública do Estado do Paraná para que possa ser analisado juntamente com os dados do estado paraens.

Nessa versão, são utilizados quatro arquivos do tipo CSV e para manipulação de dados é utilizado o pacote pyspark.

**Observação:** O código contido nesse notebook foi baseado na solução desenvolvida por Mariana Kniss (https://www.linkedin.com/in/mariana-kniss-471ba0196/), para validar o conteúdo de seu trabalho de conclusão de curso, intitulado "ANÁLISE DE DADOS CRIMINAIS SOBRE VIOLÊNCIA CONTRA A MULHER NOS ESTADOS PARÁ E PARANÁ".

## Sumário

1. [Importação dos módulos e pacotes necessários](#importacao-dos-modulos-e-pacotes-necessarios)
2. [Carregamento dos dados](#carregamento-dos-dados)
4. [Visão geral do conjunto de dados](#visao-geral-do-conjunto-de-dados)
5. [Filtros, tratamentos e padronização de valores](#filtros-tratamentos-e-padronizacao-de-valores)
6. [Ordenação das colunas](#ordernacao-das-colunas)
7. [Exportação dos dados formatados](#exportacao-dos-dados-formatados)

----


## Importação dos módulos e pacotes necessários <a id="importacao-dos-modulos-e-pacotes-necessarios"></a>

In [ ]:
# pySpark (https://spark.apache.org/docs/latest/api/python/index.html)
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import col, lower, when, substring, hour, to_timestamp, date_format, count, current_date, datediff, mean, median, variance, stddev,  collect_list, first, regexp_replace, lit, split, upper
from pyspark.sql.types import StringType, StructType, StructField, DateType
from pyspark.ml import stat

In [ ]:
# Criando sessão spark
spark = SparkSession.builder\
    .master('local[*]')\
    .appName('transformacao_VCM_PR')\
    .getOrCreate()

## Carregamento dos dados

In [ ]:
# Dados sobre as ocorrências
ocorrencias = spark.read.csv('./dados/PR/Ocorrencias de ViolDom ContraMulher (Jan2018-Ago2023) PR.csv',
                       sep = ';',
                       inferSchema= True,
                       header = True)

In [ ]:
# Dados sobre as naturezas
naturezas = spark.read.csv('./dados/PR/Naturezas das Ocorrencias de ViolDom ContraMulher (Jan2018-Ago2023) PR.csv',
                       sep = ';',
                       inferSchema= True,
                       header = True)

In [ ]:
# Dados sobre as vítimas
vitimas = spark.read.csv('./dados/PR/Vitimas das Ocorrencias de ViolDom ContraMulher (Jan2018-Ago2023) PR.csv',
                       sep = ';',
                       inferSchema= True,
                       header = True)

In [ ]:
# Dados sobre os autores
autores = spark.read.csv('./dados/PR/Autores das Ocorrencias de ViolDom ContraMulher (Jan2018-Ago2023) PR.csv',
                       sep = ';',
                       inferSchema= True,
                       header = True)

Contagem dos registros carregados.

In [ ]:
print(f'Foram encontradas {ocorrencias.count()} ocorrências.')
print(f'Foram encontradas {naturezas.select("id_boletim").distinct().count()} naturezas.')
print(f'Foram encontradas {vitimas.select("id_boletim").distinct().count()} vítimas.')
print(f'Foram encontradas {autores.select("id_boletim").distinct().count()} autores.')



## Agrupamento dos dados

In [ ]:
agrupado_df = naturezas.groupBy("id_boletim").agg(collect_list("registros").alias("registros_unicos"))
agrupado_df.show(truncate=False)

In [ ]:
agrupado_df.count()

In [ ]:
naturezas.select("registros").distinct().count()

In [ ]:
naturezas.select("classe_motivo").distinct().count()

In [ ]:
naturezas.select("classe_motivo").distinct().show(truncate=False)

In [ ]:
naturezas_feminicidio = naturezas.filter(lower(col('registros')).like('feminicidio'))
naturezas_feminicidio.count() 

In [ ]:
# Selecionando a base de VCM com base na variavel 'classe_motivo', utilizando como filtro as palavras chave
palavras_chave = ["mulher", "sexual", "medidas protetivas", "13772"]
naturezas = naturezas.filter(lower(col("classe_motivo")).like("%" + palavras_chave[0] + "%")
                                   | lower(col("classe_motivo")).like("%" + palavras_chave[1] + "%")
                                   | lower(col("classe_motivo")).like("%" + palavras_chave[2] + "%")
                                   | lower(col("classe_motivo")).like("%" + palavras_chave[3] + "%"))

# Exibindo o dataset VCM
naturezas.show()

In [ ]:
naturezas.count()

In [ ]:
naturezas.select("id_boletim").distinct().count()

In [ ]:
len(naturezas.columns)

In [ ]:
naturezas_unique = naturezas.groupBy("id_boletim").agg(
    first("data_registro").alias("data_registro"),
    first("hora_registro").alias("hora_registro"),
    first("data_fato").alias("data_fato"),
    first("dia_semana").alias("dia_semana"),
    first("hora_fato").alias("hora_fato"),
    first("fx_4_hr").alias("fx_4_hr"),
    first("fx_12_hr").alias("fx_12_hr"),
    first("mes_registro").alias("mes_registro"),
    first("mes_fato").alias("mes_fato"),
    first("ano_registro").alias("ano_registro"),
    first("ano_fato").alias("ano_fato"),
    first("causa_presumivel").alias("causa_presumivel"),
    first("distrito").alias("distrito"),
    first("municipio").alias("municipio"),
    first("regional").alias("regional"),
    first("bairro").alias("bairro"),
    first("identificacao_fato").alias("identificacao_fato"),
    first("local_ocorrencia").alias("local_ocorrencia"),
    first("meio_empregado").alias("meio_empregado"),
    first("classe_motivo").alias("classe_motivo"),
    collect_list("registros").alias("registros_unicos")
)

In [ ]:
naturezas_unique.show()

In [ ]:
len(naturezas_unique.columns)

In [ ]:
naturezas_unique.count()

In [ ]:
naturezas_unique.printSchema()

In [ ]:
from pyspark.sql.functions import concat_ws

# # Crie uma sessão do Spark
# spark = SparkSession.builder.appName("exemplo").getOrCreate()

# # Suponha que você tenha um DataFrame chamado df com uma coluna chamada 'coluna_array'
# # que contém arrays que você deseja converter para strings

# # Exemplo de dados
# dados = [("linha1", [1, 2, 3]),
#          ("linha2", [4, 5, 6]),
#          ("linha3", [7, 8, 9])]

# # Defina o esquema do DataFrame
# esquema = ["id", "coluna_array"]

# # Crie o DataFrame
# df = spark.createDataFrame(dados, esquema)

# Use a função concat_ws para converter a coluna de array para string
df_resultado = naturezas_unique.withColumn("registros", concat_ws(",", naturezas_unique["registros_unicos"]))

# Mostre o resultado
df_resultado.show()

In [ ]:
df_resultado.printSchema()

In [ ]:
df_resultado.count()

In [ ]:
len(df_resultado.columns)

In [ ]:
naturezas_unique = df_resultado.drop('registros_unicos')

In [ ]:
len(naturezas_unique.columns)

In [ ]:
naturezas_feminicidio.printSchema()

In [ ]:
# Use a função 'union' para unir os DataFrames
naturezas_union = naturezas_unique.union(naturezas_feminicidio)

# Exiba o DataFrame resultante
naturezas_union.count()

In [ ]:
naturezas_unique = naturezas_union

In [ ]:
naturezas_unique.select("classe_motivo").distinct().count()

In [ ]:
#naturezas_unique.select("registros_unicos").distinct().count()

In [ ]:
naturezas_unique.select("classe_motivo").distinct().show(truncate=False)

In [ ]:
#agrupando os dados por registros_unicos, mostrando todas as categorias possiveis
#naturezas_unique.groupBy("registros_unicos").count().orderBy(col("count").desc()).show(naturezas_unique.count(), truncate=False)

In [ ]:
naturezas_nao_unicos = naturezas.groupBy("id_boletim").count().orderBy(col("id_boletim").desc())
naturezas_nao_unicos.filter(naturezas_nao_unicos["count"] == 3).show()

In [ ]:
naturezas.filter(naturezas['id_boletim'] == "20201001697").show(truncate=False)

In [ ]:
naturezas_unique.filter(naturezas_unique['id_boletim'] == "20201001697").show(truncate=False)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Combine todos os valores de 'registros_unicos' em uma única string
texto_registros = " ".join(naturezas_unique.rdd.flatMap(lambda x: x["registros_unicos"]).collect())

# Crie a nuvem de palavras
nuvem_palavras = WordCloud(width=800, height=400, background_color='white').generate(texto_registros)

# Exiba a nuvem de palavras
plt.figure(figsize=(10, 5))
plt.imshow(nuvem_palavras, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
registros = " ".join(naturezas.select("registros").rdd.flatMap(lambda x: x).collect())

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(registros)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
naturezas = naturezas.withColumn("registros", lower(regexp_replace(col("registros"), "[^a-zA-Z ]", "")))

contagem_judicial = naturezas.filter(naturezas.registros.contains("judicial")).count()
contagem_estupro = naturezas.filter(naturezas.registros.contains("estupro")).count()

print("Número de ocorrências da palavra 'judicial':", contagem_judicial)
print("Número de ocorrências da palavra 'estupro':", contagem_estupro)


In [ ]:
cloud = naturezas.filter(
    (naturezas.registros != 'DESCUMPRIR DECISAO JUDICIAL QUE DEFERE MEDIDAS PROTETIVAS DE URGENCIA') &
    (naturezas.registros != 'DESCUMPRIR DECISAO JUDICIAL QUE DEFERE MEDIDAS PROTETIVAS DE URGENCIA PREVISTAS NESTA LEI') &
    (naturezas.registros != 'ESTA LEI CRIA MECANISMOS PARA COIBIR E PREVENIR A ')
)

In [ ]:
registros = " ".join(cloud.select("registros").rdd.flatMap(lambda x: x).collect())

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(registros)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## Tentativas de JOIN

In [ ]:
ocorrencias.show()

In [ ]:
len(ocorrencias.columns)

In [ ]:
vitimas.show()

In [ ]:
len(vitimas.columns)

In [ ]:
dados = vitimas.join(naturezas.select('id_boletim', 'local_ocorrencia', 'meio_empregado', 'classe_motivo', 'registros'), 'id_boletim', 'left')

In [ ]:
len(dados.columns)

In [ ]:
dados.show()

In [ ]:
dados.select('id_boletim').distinct().count()

In [ ]:
dados.count()

In [ ]:
dados = dados.join(naturezas.select('id_boletim', 'local_ocorrencia', 'meio_empregado', 'classe_motivo', 'registros'), 'id_boletim', 'left')

In [ ]:
dados.count()

In [ ]:
dados = dados.join(autores.select('id_boletim', 'autor', 'aut_dt_nasc', 'aut_idade', 'faixa_etaria', 'parentesco', 'sexo', 'raca_cor', 'grau_instrucao', 'profissao'), 'id_boletim', 'outer')

In [ ]:
dados.count()

In [ ]:
dados_VCM = ocorrencias.join(vitimas.select('id_boletim', 'vitima', 'vit_dt_nasc', 'vit_idade', 'faixa_etaria', 'sexo', 'raca_cor', 'grau_instrucao', 'profissao'), 'id_boletim', 'outer')

In [ ]:
dados_VCM.count()

In [ ]:
dados_VCM = dados_VCM.join(naturezas.select('id_boletim', 'local_ocorrencia', 'meio_empregado', 'classe_motivo', 'registros'), 'id_boletim', 'left')

In [ ]:
dados_VCM.show()

## Novas tentativas de JOIN

In [ ]:
naturezas.count()

In [ ]:
naturezas_unique.count()

In [ ]:
naturezas_vitimas = naturezas_unique.join(vitimas.select('id_boletim', 'vitima', 'vit_dt_nasc', 'vit_idade', 'faixa_etaria', 'raca_cor', 'grau_instrucao', 'profissao'), on='id_boletim')

In [ ]:
naturezas_vitimas.count()

In [ ]:
naturezas_vitimas.select("id_boletim").distinct().count()

In [ ]:
naturezas_vitimas.show()

In [ ]:
autores = autores \
    .withColumnRenamed("faixa_etaria", "aut_faixa_etaria") \
    .withColumnRenamed("raca_cor", "aut_raca_cor") \
    .withColumnRenamed("grau_instrucao", "aut_grau_instrucao") \
    .withColumnRenamed("profissao", "aut_profissao")

In [ ]:
naturezas_vitimas_autores = naturezas_vitimas.join(autores.select('id_boletim', 'autor', 'aut_dt_nasc', 'aut_idade', 'aut_faixa_etaria', 'aut_raca_cor', 'aut_grau_instrucao', 'aut_profissao', 'parentesco'), on='id_boletim', how='left')

In [ ]:
naturezas_vitimas_autores.count()

In [ ]:
naturezas_vitimas_autores.show()

In [ ]:
colunas_remover = ['id_boletim', 'fx_4_hr', 'fx_12_hr', 'hora_registro', 'hora_fato', 'causa_presumivel', 'distrito', 'regional', 'vitima']
dados_VCM_PR = naturezas_vitimas_autores.drop(*colunas_remover)

In [ ]:
dados_VCM_PR.count()

In [ ]:
dados_VCM_PR = dados_VCM_PR.withColumn("estado", lit("PR"))

In [ ]:
dados_VCM_PR.show()

In [ ]:
# Duplica as colunas 'data_registro' e 'data_fato' como 'data_registro_dup' e 'data_fato_dup'
dados_VCM_PR = dados_VCM_PR.withColumn("data_registro_dup", col("data_registro"))
dados_VCM_PR = dados_VCM_PR.withColumn("data_fato_dup", col("data_fato"))

In [ ]:
# Divida as colunas 'data_registro_dup' e 'data_fato_dup' em duas colunas: data e hora
dados_VCM_PR = dados_VCM_PR.withColumn("data_registro", split(col("data_registro"), " ").getItem(0))
dados_VCM_PR = dados_VCM_PR.withColumn("hora_registro", split(col("data_registro_dup"), " ").getItem(1))
dados_VCM_PR = dados_VCM_PR.withColumn("data_fato", split(col("data_fato"), " ").getItem(0))
dados_VCM_PR = dados_VCM_PR.withColumn("hora_fato", split(col("data_fato_dup"), " ").getItem(1))

In [ ]:
dados_VCM_PR.show()

In [ ]:
colunas_remover = ['data_registro_dup', 'data_fato_dup']
dados_VCM_PR = dados_VCM_PR.drop(*colunas_remover)

In [ ]:
dados_VCM_PR.show()

In [ ]:
len(dados_VCM_PR.columns)

In [ ]:
# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PR.columns
colunas_reorganizadas = colunas[:2] + [colunas[30]] + colunas[2:30]

# Selecione as colunas na ordem desejada
dados_VCM_PR_ordenadas = dados_VCM_PR.select(colunas_reorganizadas)

# Exiba o DataFrame resultante
dados_VCM_PR_ordenadas.show()

In [ ]:
# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PR_ordenadas.columns
colunas_reorganizadas = colunas[:1] + [colunas[30]] + colunas[1:30]

# Selecione as colunas na ordem desejada
dados_VCM_PR_ordenadas = dados_VCM_PR_ordenadas.select(colunas_reorganizadas)

# Exiba o DataFrame resultante
dados_VCM_PR_ordenadas.show()

In [ ]:
dados_VCM_PR = dados_VCM_PR_ordenadas

In [ ]:
dados_VCM_PR = dados_VCM_PR.filter(dados_VCM_PR['ano_fato'] < 2023)

In [ ]:
dados_VCM_PR.groupBy('ano_fato').count().show()

In [ ]:
dados_VCM_PR.count()

In [ ]:
dados_VCM_PR = dados_VCM_PR.withColumn("dia_semana", upper(dados_VCM_PR["dia_semana"]))

In [ ]:
dados_VCM_PR.show()

In [ ]:
# Use a função 'when' para criar a coluna 'faixa_horaria' com base na coluna 'hora_fato'
dados_VCM_PR = dados_VCM_PR.withColumn("faixa_horaria",
                                       when((dados_VCM_PR['hora_fato'] >= '06:00:00') & (dados_VCM_PR['hora_fato'] < '12:00:00'), 'Manhã')
                                      .when((dados_VCM_PR['hora_fato'] >= '12:00:00') & (dados_VCM_PR['hora_fato'] < '18:00:00'), 'Tarde')
                                      .when((dados_VCM_PR['hora_fato'] >= '18:00:00') & (dados_VCM_PR['hora_fato'] < '23:59:59'), 'Noite')
                                      .when((dados_VCM_PR['hora_fato'] >= '00:00:00') & (dados_VCM_PR['hora_fato'] < '06:00:00'), 'Madrugada')
                                      .otherwise('Outro'))

# Exiba o DataFrame resultante
dados_VCM_PR.show()

In [ ]:
# Use a função 'when' para criar a coluna 'faixa_horaria' com base na coluna 'hora_fato'
dados_VCM_PR = dados_VCM_PR.withColumn("faixa_horaria_6h",
                                       when((dados_VCM_PR['hora_fato'] >= '06:00:00') & (dados_VCM_PR['hora_fato'] < '12:00:00'), '06h - 12h')
                                      .when((dados_VCM_PR['hora_fato'] >= '12:00:00') & (dados_VCM_PR['hora_fato'] < '18:00:00'), '12h - 18h')
                                      .when((dados_VCM_PR['hora_fato'] >= '18:00:00') & (dados_VCM_PR['hora_fato'] < '23:59:59'), '18h - 00h')
                                      .when((dados_VCM_PR['hora_fato'] >= '00:00:00') & (dados_VCM_PR['hora_fato'] < '06:00:00'), '00h - 06h')
                                      .otherwise('Outro'))

# Exiba o DataFrame resultante
dados_VCM_PR.show()

In [ ]:
# Use a função 'when' para criar a coluna 'faixa_horaria_2h' com base na coluna 'hora_fato'
dados_VCM_PR = dados_VCM_PR.withColumn("faixa_horaria_2h",
                                       when((dados_VCM_PR['hora_fato'] >= '06:00:00') & (dados_VCM_PR['hora_fato'] < '08:00:00'), '06h - 08h')
                                      .when((dados_VCM_PR['hora_fato'] >= '08:00:00') & (dados_VCM_PR['hora_fato'] < '10:00:00'), '08h - 10h')
                                      .when((dados_VCM_PR['hora_fato'] >= '10:00:00') & (dados_VCM_PR['hora_fato'] < '12:00:00'), '10h - 12h')
                                      .when((dados_VCM_PR['hora_fato'] >= '12:00:00') & (dados_VCM_PR['hora_fato'] < '14:00:00'), '12h - 14h')
                                      .when((dados_VCM_PR['hora_fato'] >= '14:00:00') & (dados_VCM_PR['hora_fato'] < '16:00:00'), '14h - 16h')
                                      .when((dados_VCM_PR['hora_fato'] >= '16:00:00') & (dados_VCM_PR['hora_fato'] < '18:00:00'), '16h - 18h')
                                      .when((dados_VCM_PR['hora_fato'] >= '18:00:00') & (dados_VCM_PR['hora_fato'] < '20:00:00'), '18h - 20h')
                                      .when((dados_VCM_PR['hora_fato'] >= '20:00:00') & (dados_VCM_PR['hora_fato'] < '22:00:00'), '20h - 22h')
                                      .when((dados_VCM_PR['hora_fato'] >= '22:00:00') & (dados_VCM_PR['hora_fato'] < '23:59:59'), '22h - 00h')
                                      .when((dados_VCM_PR['hora_fato'] >= '00:00:00') & (dados_VCM_PR['hora_fato'] < '02:00:00'), '00h - 02h')
                                      .when((dados_VCM_PR['hora_fato'] >= '02:00:00') & (dados_VCM_PR['hora_fato'] < '04:00:00'), '02h - 04h')
                                      .when((dados_VCM_PR['hora_fato'] >= '04:00:00') & (dados_VCM_PR['hora_fato'] < '06:00:00'), '04h - 06h')
                                      .otherwise('Outro'))

# Exiba o DataFrame resultante
dados_VCM_PR.show()

In [ ]:
dados_VCM_PR.count()

In [ ]:
dados_VCM_PR.count()

In [ ]:
dados_VCM_PR.printSchema()

In [ ]:
#selecionando e tipando a data das variaveis
dados_VCM_PR = dados_VCM_PR.withColumn("data_registro", when(col("data_registro").isNotNull(), col("data_registro").cast(DateType())))
dados_VCM_PR = dados_VCM_PR.withColumn("data_fato", when(col("data_fato").isNotNull(), col("data_fato").cast(DateType())))
dados_VCM_PR = dados_VCM_PR.withColumn("vit_dt_nasc", when(col("vit_dt_nasc").isNotNull(), col("vit_dt_nasc").cast(DateType())))
dados_VCM_PR = dados_VCM_PR.withColumn("aut_dt_nasc", when(col("aut_dt_nasc").isNotNull(), col("aut_dt_nasc").cast(DateType())))

In [ ]:
dados_VCM_PR.printSchema()

In [ ]:
# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PR.columns
colunas_reorganizadas = colunas[:4] + [colunas[33]] + colunas[4:33]

# Selecione as colunas na ordem desejada
dados_VCM_PR_ordenadas = dados_VCM_PR.select(colunas_reorganizadas)

# Exiba o DataFrame resultante
dados_VCM_PR_ordenadas.show()

In [ ]:
# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PR_ordenadas.columns
colunas_reorganizadas = colunas[:5] + [colunas[33]] + colunas[5:33]

# Selecione as colunas na ordem desejada
dados_VCM_PR_ordenadas = dados_VCM_PR_ordenadas.select(colunas_reorganizadas)

# Exiba o DataFrame resultante
dados_VCM_PR_ordenadas.show()

In [ ]:
# Use a lista com as colunas reorganizadas
colunas = dados_VCM_PR_ordenadas.columns
colunas_reorganizadas = colunas[:6] + [colunas[33]] + colunas[6:33]

# Selecione as colunas na ordem desejada
dados_VCM_PR_ordenadas = dados_VCM_PR_ordenadas.select(colunas_reorganizadas)

# Exiba o DataFrame resultante
dados_VCM_PR_ordenadas.show()

In [ ]:
dados_VCM_PR = dados_VCM_PR_ordenadas

In [ ]:
# Calcule a idade em anos com tratamento para valores nulos
dados_VCM_PR_cal_idade = dados_VCM_PR.withColumn("vit_idade",
                   when(
                       (dados_VCM_PR["data_fato"].isNotNull()) & (dados_VCM_PR["vit_dt_nasc"].isNotNull()),
                       (datediff(dados_VCM_PR["data_fato"], dados_VCM_PR["vit_dt_nasc"]) / 365).cast("int")
                   ).otherwise(None))

In [ ]:
# Calcule a idade em anos com tratamento para valores nulos
dados_VCM_PR_cal_idade = dados_VCM_PR_cal_idade.withColumn("aut_idade",
                   when(
                       (dados_VCM_PR_cal_idade["data_fato"].isNotNull()) & (dados_VCM_PR_cal_idade["aut_dt_nasc"].isNotNull()),
                       (datediff(dados_VCM_PR_cal_idade["data_fato"], dados_VCM_PR_cal_idade["aut_dt_nasc"]) / 365).cast("int")
                   ).otherwise(None))

In [ ]:
dados_VCM_PR = dados_VCM_PR_cal_idade

In [ ]:
dados_VCM_PR.count()

In [ ]:
len(dados_VCM_PR.columns)

In [ ]:
colunas_remover = ['identificacao_fato', 'autor']
dados_VCM_PR = dados_VCM_PR.drop(*colunas_remover)

In [ ]:
dados_VCM_PR = dados_VCM_PR \
    .withColumnRenamed("faixa_etaria", "vit_faixa_etaria") \
    .withColumnRenamed("raca_cor", "vit_raca_cor") \
    .withColumnRenamed("grau_instrucao", "vit_grau_instrucao") \
    .withColumnRenamed("profissao", "vit_profissao")

In [ ]:
dados_VCM_PR = dados_VCM_PR \
    .withColumnRenamed("vit_raca_cor", "vit_cor_pele") \
    .withColumnRenamed("aut_raca_cor", "aut_cor_pele") \
    .withColumnRenamed("parentesco", "grau_de_relacionamento") \
    .withColumnRenamed("vit_grau_instrucao", "vit_grau_inst") \
    .withColumnRenamed("aut_grau_instrucao", "aut_grau_inst")

In [ ]:
# Use a função 'when' para criar uma nova coluna com as faixas etárias
dados_VCM_PR = dados_VCM_PR.withColumn(
    "vit_faixa_etaria",
    when((dados_VCM_PR.vit_idade >= 0) & (dados_VCM_PR.vit_idade <= 11), "CRIANCA (0 A 11 ANOS)")
    .when((dados_VCM_PR.vit_idade >= 12) & (dados_VCM_PR.vit_idade <= 17), "ADOLESCENTE (12 A 17 ANOS)")
    .when((dados_VCM_PR.vit_idade >= 18) & (dados_VCM_PR.vit_idade <= 24), "ADULTO I (18 A 24 ANOS)")
    .when((dados_VCM_PR.vit_idade >= 25) & (dados_VCM_PR.vit_idade <= 29), "ADULTO II (25 A 29 ANOS)")
    .when((dados_VCM_PR.vit_idade >= 30) & (dados_VCM_PR.vit_idade <= 34), "ADULTO III (30 A 34 ANOS)")
    .when((dados_VCM_PR.vit_idade >= 35) & (dados_VCM_PR.vit_idade <= 64), "ADULTO IV (35 A 64 ANOS)")
    .when(dados_VCM_PR.vit_idade >= 65, "IDOSO")
    .otherwise("Desconhecida")  # Lidando com vit_idades fora das faixas
)

In [ ]:
# Use a função 'when' para criar uma nova coluna com as faixas etárias
dados_VCM_PR = dados_VCM_PR.withColumn(
    "aut_faixa_etaria",
    when((dados_VCM_PR.aut_faixa_etaria >= 0) & (dados_VCM_PR.aut_faixa_etaria <= 11), "CRIANCA (0 A 11 ANOS)")
    .when((dados_VCM_PR.aut_faixa_etaria >= 12) & (dados_VCM_PR.aut_faixa_etaria <= 17), "ADOLESCENTE (12 A 17 ANOS)")
    .when((dados_VCM_PR.aut_faixa_etaria >= 18) & (dados_VCM_PR.aut_faixa_etaria <= 24), "ADULTO I (18 A 24 ANOS)")
    .when((dados_VCM_PR.aut_faixa_etaria >= 25) & (dados_VCM_PR.aut_faixa_etaria <= 29), "ADULTO II (25 A 29 ANOS)")
    .when((dados_VCM_PR.aut_faixa_etaria >= 30) & (dados_VCM_PR.aut_faixa_etaria <= 34), "ADULTO III (30 A 34 ANOS)")
    .when((dados_VCM_PR.aut_faixa_etaria >= 35) & (dados_VCM_PR.aut_faixa_etaria <= 64), "ADULTO IV (35 A 64 ANOS)")
    .when(dados_VCM_PR.aut_faixa_etaria >= 65, "IDOSO")
    .otherwise("Desconhecida")  # Lidando com aut_faixa_etarias fora das faixas
)

In [ ]:
# Use a função 'withColumn' para alterar o tipo das colunas de integer para string
dados_VCM_PR = dados_VCM_PR.withColumn("mes_registro", col("mes_registro").cast("string"))
dados_VCM_PR = dados_VCM_PR.withColumn("mes_fato", col("mes_fato").cast("string"))
dados_VCM_PR = dados_VCM_PR.withColumn("ano_registro", col("ano_registro").cast("string"))
dados_VCM_PR = dados_VCM_PR.withColumn("ano_fato", col("ano_fato").cast("string"))

# Exiba o DataFrame resultante
dados_VCM_PR.show()

In [ ]:
dados_VCM_PR.printSchema()

In [ ]:
dados_VCM_PR.count()

In [ ]:
dados_VCM_pd = dados_VCM_PR.toPandas()
dados_VCM_pd.to_csv("./dados-tratados/dados_VCM_PR.csv", index=False, sep='|', encoding='utf-8')